In [1]:
from config.DatabaseConfig import *
from config.GlobalParams import *

In [2]:
from utils.Database import Database
from utils.Preprocess import Preprocess

In [3]:
# 전처리 객체 생성
p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dict.bin', userdic='./utils/user_dic.tsv')

# 질문/답변 학습 디비 연결 객체 생성
#db = Database(
#    host="127.0.0.1", user="root", password="    ", db_name="homestead"
#)


db = Database(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        db_name=DB_NAME,
        charset='utf8'
)
db.connect()    # 디비 연결

In [4]:
# 원문
#query = "오전에 탕수육 10개 주문합니다"
#query = "화자의 질문 의도를 파악합니다."
# query = "안녕하세요"
#query = "자장면 주문할게요"
query = "이하상게 한인국은 읽수을가 있든거요"

# 의도 파악
from models.intent.IntentModel import IntentModel

intent = IntentModel(model_name='./models/intent/intent_model.h5', proprocess=p)
predict = intent.predict_class(query)
intent_name = intent.labels[predict]

# 개체명 인식
from models.ner.NerModel import NerModel
ner = NerModel(model_name='./models/ner/ner_model.h5', proprocess=p)
predicts = ner.predict(query)
ner_tags = ner.predict_tags(query)

print("질문 : ", query)
print("=" * 100)
print("의도 파악 : ", intent_name)
print("개체명 인식 : ", predicts)
print("답변 검색에 필요한 NER 태그 : ", ner_tags)

질문 :  이하상게 한인국은 읽수을가 있든거요
의도 파악 :  기타
개체명 인식 :  [('이', 'O'), ('하상', 'O'), ('게', 'O'), ('한인', 'O'), ('국', 'O'), ('읽', 'O'), ('수', 'O'), ('있', 'O'), ('거', 'O')]
답변 검색에 필요한 NER 태그 :  None


In [5]:
# 답변 검색
from utils.FindAnswer import FindAnswer

try:
    f = FindAnswer(db)
    answer_text, answer_image = f.search(intent_name, ner_tags)
    answer = f.tag_to_word(predicts, answer_text)
except:
    answer = "죄송해요 무슨 말인지 모르겠어요"

print("답변 : ", answer)

db.close() # 디비 연결 끊음


답변 :  죄송해요 무슨 말인지 모르겠어요
